In [13]:
!pip install geoplot
!pip install geojsonio

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 151.8/151.8 kB 302.5 kB/s eta 0:00:00
  Created wheel for geojsonio: filename=geojsonio-0.0.3-py3-none-any.whl size=5024 sha256=40550e5fe7a224f5d12a7f6616087231fa82a2488b34d38076ab13e8f5ce303e
  Stored in directory: c:\users\nisleida morales\appdata\local\pip\cache\wheels\5b\53\d5\01006178d29712c91b7917f736192c5bb5fcf6e4a06816b903
Successfully built geojsonio


In [29]:
import geopandas as gpd
import folium

In [23]:
road_alignment = gpd.read_file('./road_alignment.geojson')

In [24]:
road_alignment.crs

<Derived Projected CRS: EPSG:32636>
Name: WGS 84 / UTM zone 36N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 30°E and 36°E, northern hemisphere between equator and 84°N, onshore and offshore. Belarus. Cyprus. Egypt. Ethiopia. Finland. Israel. Jordan. Kenya. Lebanon. Moldova. Norway. Russian Federation. Saudi Arabia. Sudan. Syria. Turkey. Uganda. Ukraine.
- bounds: (30.0, 0.0, 36.0, 84.0)
Coordinate Operation:
- name: UTM zone 36N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
# Create a Folium map centered on the line
center_lat = road_alignment['geometry'].centroid.y.mean()
center_lon = road_alignment['geometry'].centroid.x.mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add GeoJSON line to the map
folium.GeoJson(road_alignment).add_to(m)

In [27]:
m